In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.decomposition import PCA, KernelPCA
import optuna

import sys
sys.path.append("../") # go to parent dir

from src.debiasing import DebiasingPCA
from src.debiasing import DebiasingKernelPCA
from src.debiasing.utils import get_design_matrix

In [2]:
data = pd.read_csv("../data/CoLA/mixed_dev_embeddings.tsv", sep="\t")
data = data.sort_values(by='defining set id')
X = data.iloc[:, 3:771].to_numpy()
X_index = data['defining set id'].to_numpy()

In [3]:
print("the shape of X:", X.shape)
X

the shape of X: (640, 768)


array([[-0.851718  , -0.4793546 , -0.77166116, ..., -0.5383287 ,
        -0.76654   ,  0.9239464 ],
       [-0.84405816, -0.49862942, -0.7877743 , ..., -0.57809174,
        -0.728355  ,  0.9136135 ],
       [-0.87432224, -0.59713227, -0.9513105 , ..., -0.40138096,
        -0.7750305 ,  0.8731641 ],
       ...,
       [-0.886508  , -0.51329815, -0.94088244, ..., -0.75856566,
        -0.7137282 ,  0.9045762 ],
       [-0.70865357, -0.30252996, -0.819484  , ..., -0.36328802,
        -0.6049608 ,  0.7728163 ],
       [-0.7912138 , -0.4454564 , -0.92030555, ..., -0.6560643 ,
        -0.64869297,  0.8218555 ]])

In [4]:
print("the shape of X_ index:", X_index.shape)
X_index

the shape of X_ index: (640,)


array([  0,   0,   1,   1,   2,   2,   3,   3,   4,   4,   5,   5,   6,
         6,   7,   7,   8,   8,   9,   9,  10,  10,  11,  11,  12,  12,
        13,  13,  14,  14,  15,  15,  16,  16,  17,  17,  18,  18,  19,
        19,  20,  20,  21,  21,  22,  22,  23,  23,  24,  24,  25,  25,
        26,  26,  27,  27,  28,  28,  29,  29,  30,  30,  31,  31,  32,
        32,  33,  33,  34,  34,  35,  35,  36,  36,  37,  37,  38,  38,
        39,  39,  40,  40,  41,  41,  42,  42,  43,  43,  44,  44,  45,
        45,  46,  46,  47,  47,  48,  48,  49,  49,  50,  50,  51,  51,
        52,  52,  53,  53,  54,  54,  55,  55,  56,  56,  57,  57,  58,
        58,  59,  59,  60,  60,  61,  61,  62,  62,  63,  63,  64,  64,
        65,  65,  66,  66,  67,  67,  68,  68,  69,  69,  70,  70,  71,
        71,  72,  72,  73,  73,  74,  74,  75,  75,  76,  76,  77,  77,
        78,  78,  79,  79,  80,  80,  81,  81,  82,  82,  83,  83,  84,
        84,  85,  85,  86,  86,  87,  87,  88,  88,  89,  89,  9

In [5]:
# data visualization
eigvals = PCA().fit(X).explained_variance_
explained_var = np.cumsum(eigvals) / np.sum(eigvals)
plt.plot(explained_var)
plt.ylabel('explained variance')
plt.xlabel('the number of components')

Text(0.5, 0, 'the number of components')

# PCA debiasing

In [6]:
debiaser = DebiasingPCA(n_components=3)
debiaser.fit(X, X_index)
X_debiased = debiaser.debias(X)

In [7]:
np.linalg.norm(X_debiased)

327.8692835758318

# Kernel PCA debiasing (by kernel ridge pre-imaging)

In [8]:
np.linalg.norm(X)

426.1726765040533

### Unsuccessful case (Learn pre-image of bias subspace by min || Phi(w) - Proj (Phi(w))||)

In [9]:
debiaser = DebiasingKernelPCA(n_components=3, kernel="rbf")
debiaser.fit(X, X_index)
X_debiased = debiaser.debias(X)

In [10]:
np.linalg.norm(X_debiased)

63.74915566837997

### Unsuccessful case (Learn pre-image of bias subspace by min || Phi(w - m) - Proj (Phi(w - m))||)

In [11]:
DX = get_design_matrix(X_index)
X_diff = DX @ X
kpca = KernelPCA(n_components=3, kernel="rbf", gamma=0.14, fit_inverse_transform=True)
kpca.fit(X_diff)
X_bias = kpca.inverse_transform(kpca.transform(X))
X_debiased = X - X_bias

In [12]:
np.linalg.norm(X_debiased)

425.8716657453607

In [13]:
# gamma = 0.14 seems to work well

def objective(trial):
    # n = trial.suggest_int("n_components", 1, 30)
    gamma = trial.suggest_loguniform('gamma', 1e-6, 1e+3)
    kpca = KernelPCA(n_components=3, kernel="rbf", gamma=gamma, fit_inverse_transform=True)
    kpca.fit(X_diff[:400])
    X_reproj = kpca.inverse_transform(kpca.transform(X_diff[400:]))
    return np.linalg.norm(X_diff[400:] - X_reproj)

study = optuna.create_study()
study.optimize(objective, n_trials=100)
print(study.best_params)

[I 2020-11-26 16:44:06,378] A new study created in memory with name: no-name-559ea537-b899-454b-9ef8-d98a2f9c8ec2
[I 2020-11-26 16:44:06,511] Trial 0 finished with value: 18.21774293838169 and parameters: {'gamma': 0.0019845533602015878}. Best is trial 0 with value: 18.21774293838169.
[I 2020-11-26 16:44:06,647] Trial 1 finished with value: 18.242141461103557 and parameters: {'gamma': 4.534603623190515e-06}. Best is trial 0 with value: 18.21774293838169.
[I 2020-11-26 16:44:06,805] Trial 2 finished with value: 18.20136362490829 and parameters: {'gamma': 63.595695684480205}. Best is trial 2 with value: 18.20136362490829.
[I 2020-11-26 16:44:06,935] Trial 3 finished with value: 18.2233949553907 and parameters: {'gamma': 0.0017382376685706735}. Best is trial 2 with value: 18.20136362490829.
[I 2020-11-26 16:44:07,078] Trial 4 finished with value: 17.884697744195474 and parameters: {'gamma': 0.007818392724133122}. Best is trial 4 with value: 17.884697744195474.
[I 2020-11-26 16:44:07,223] 

[I 2020-11-26 16:44:14,654] Trial 44 finished with value: 10.92623024713603 and parameters: {'gamma': 0.11067855377871015}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:14,930] Trial 45 finished with value: 12.305054387271724 and parameters: {'gamma': 0.4917809778654479}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:15,179] Trial 46 finished with value: 18.050815232937733 and parameters: {'gamma': 0.005645767891880911}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:15,388] Trial 47 finished with value: 16.244074008106573 and parameters: {'gamma': 0.020929168897376657}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:15,592] Trial 48 finished with value: 14.458171256462618 and parameters: {'gamma': 2.0693949179759747}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:15,814] Trial 49 finished with value: 12.405992442689358 and parameters: {'gamma': 0.5849957467625032}. Best is

[I 2020-11-26 16:44:24,539] Trial 88 finished with value: 12.033500398413144 and parameters: {'gamma': 0.06375587075393642}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:24,778] Trial 89 finished with value: 17.4515699182932 and parameters: {'gamma': 0.012007155349465589}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:25,002] Trial 90 finished with value: 18.160395432994278 and parameters: {'gamma': 0.0036551304701002724}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:25,348] Trial 91 finished with value: 10.821589448660372 and parameters: {'gamma': 0.12735621008953943}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:25,553] Trial 92 finished with value: 12.058864737228319 and parameters: {'gamma': 0.2587922739953831}. Best is trial 21 with value: 10.781998409819579.
[I 2020-11-26 16:44:25,856] Trial 93 finished with value: 10.795036159025944 and parameters: {'gamma': 0.13557908832556503}. Best 

{'gamma': 0.1426568660664404}
